In [99]:
# Definição do problema
from pulp import *

In [100]:
def relatorio(model, variables, tipo='Minimizar'):
    # Função Objetivo
    print(f'{tipo}:')
    print(f'\tz = {model.objective}')
    print()
    # Restrições
    print('Sujeito a:')
    for restricao in model.constraints.values():
        print(f'\t{restricao}')
    print(f'\t{str(variables)[1:-1]} ∈ [0,1]')
    print()

    # Resolvendo o problema
    status = model.solve()

    # Vendo resultados
    print(f"Status: {'Sucesso' if model.status else 'Falha':}, solução { 'ótima' if LpStatus[model.status] == 'Optimal' else LpStatus[model.status]} encontrada")
    print(f"Valor objetivo (z): {round(model.objective.value(),2)}")
    print()

    print("Valores encontrados:")
    for var in variables:
        print(f"{var.name}: {var.value()}")

In [101]:
modelo = LpProblem("ExpedicaoPesquisaMarte", LpMinimize)
# Homens
homens = ["Fábio", "Gallo", "Saulo", "Paulo", "José", "Bruno"]
qtd_homens = len(homens)
# h = LpVariable.dict("h", [i for i in range(qtd_homens)], lowBound = 0, upBound = 1, cat='Integer') # variável binária
h = {i: LpVariable(name=f"h{homens[i][0]}", lowBound=0, upBound=1, cat='Integer') for i in range(qtd_homens)} # variável binária

# Mulheres
mulheres = ["Maria", "Ana", "Linda", "Dina", "Carla"]
qtd_mulheres = len(mulheres)
# m = LpVariable.dicts("m", [i for i in range(qtd_mulheres)], lowBound = 0, upBound = 1, cat='Integer') # variável binária
m = {i: LpVariable(name=f"m{mulheres[i][0]}", lowBound=0, upBound=1, cat='Integer') for i in range(qtd_mulheres)} # variável binária


In [102]:
# a) Restrição associada ao número de homens e mulheres na missão

modelo += lpSum(h[i] for i in range(qtd_homens)) >= 2 # Quantidade mínima de homens na missão
                 
modelo += lpSum(m[i] for i in range(qtd_mulheres)) >= 1 # Quantidade mínima de mulheres na missão                 


In [103]:
# b) Restrição associada à exigência da habilidade de comando:

# Fábio, Gallo, José, Bruno e Dina possuem a habilidades

modelo += lpSum([h[0], h[1], h[4], h[5], m[3]]) >= 2

In [104]:
# c) Restrição associada à exigência da habilidade de psicologia:

# Dina e Carla possuem a habilidade

modelo += lpSum([m[3], m[4]]) >= 1

In [105]:
# d) Restrição associada à exigência da habilidade de astronomia:

# Saulo, Paulo e Bruno possuem a habilidade
modelo += lpSum([h[2], h[3], h[5]]) >= 1

In [106]:
# e) Restrição associada à exigência da habilidade de informática:

# Fábio, José, Maria, Linda, Dina possuem a habilidade

modelo += lpSum([h[homens.index("Fábio")], h[homens.index("José")], m[mulheres.index("Maria")], m[mulheres.index("Linda")], m[mulheres.index("Dina")]]) >= 1

In [107]:
# f) Restrição associada ao uso do simulador:

# homens = ["Fábio", "Gallo", "Saulo", "Paulo", "José", "Bruno"]
# mulheres = ["Maria", "Ana", "Linda", "Dina", "Carla"]

modelo += lpSum([15*h[0], 20*h[1], 18*h[2], 24*h[3], 22*h[4], 10*h[5], 20*m[0], 25*m[1], 20*m[2], 18*m[3], 10*m[3]]) <= 1000

In [108]:
# g) Restrição de compatibilidade pessoal:

modelo += lpSum([h[homens.index("Fábio")], h[homens.index("José")]]) <= 1
modelo += lpSum([h[homens.index("Bruno")], m[mulheres.index("Linda")]]) <= 1
modelo += lpSum([m[mulheres.index("Dina")], m[mulheres.index("Ana")]]) <= 1

In [109]:
# POR ENQUANTO ESTÁ COMPLEXA E SERÁ REMOVIDO

# # h) Restrição associada aos aspectos epidemiológicos:

# saude_homens = [150, 150, 95, 140, 90, 105]
# saude_mulheres = [145, 145, 105, 140, 95]

# combinacoes = {"homens": saude_homens, "mulheres": saude_mulheres}
# combinacao = pulp.LpVariable.dicts("combine", combinacoes, cat='Binary')

# h) Quantidade de pessoas na expedição

modelo += lpSum([h[i] for i in range(qtd_homens)] + [m[j] for j in range(qtd_mulheres)]) == 5


In [110]:
peso_homens = [80, 85, 70, 65, 90, 75]
baguagem_homens = [35, 25, 30, 40, 17, 40]

peso_homens = {homens[i]: [peso_homens[i], baguagem_homens[i]] for i in range(qtd_homens)}

peso_mulheres = [50, 45, 55, 60, 65]
baguagem_mulheres = [30, 40, 30, 20, 30]

peso_mulheres = {mulheres[i]: [peso_mulheres[i], baguagem_mulheres[i]] for i in range(qtd_mulheres)}

In [111]:
# A: É a parcela relativa ao custo pelo peso dos tripulantes e suas bagagens

A = [(peso_homens[nome][0] + peso_homens[nome][1])*h[homens.index(nome)] for nome in homens]
A += [(peso_mulheres[nome][0] + peso_mulheres[nome][1])*m[mulheres.index(nome)] for nome in mulheres]
A = sum(A)

In [112]:
# teta = LpVariable(name="teta", lowBound=0, upBound=1, cat='Integer')

# B = [(peso_homens[nome][0] + peso_homens[nome][1])*h[homens.index(nome)] for nome in homens]
# B += [(peso_mulheres[nome][0] + peso_mulheres[nome][1])*m[mulheres.index(nome)] for nome in mulheres]
# B = sum(B) - 1000

# print(B)

# modelo += (B)*teta >= 0

# modelo += (10**4)*A + (10**7)*teta

In [113]:
modelo += A

In [114]:
modelo.solve()

1

In [115]:
relatorio(modelo, modelo.variables(), 'Minimizar')

Minimizar:
	z = 115*hB + 115*hF + 110*hG + 107*hJ + 105*hP + 100*hS + 85*mA + 95*mC + 80*mD + 85*mL + 80*mM

Sujeito a:
	hB + hF + hG + hJ + hP + hS >= 2
	mA + mC + mD + mL + mM >= 1
	hB + hF + hG + hJ + mD >= 2
	mC + mD >= 1
	hB + hP + hS >= 1
	hF + hJ + mD + mL + mM >= 1
	10*hB + 15*hF + 20*hG + 22*hJ + 24*hP + 18*hS + 25*mA + 28*mD + 20*mL + 20*mM <= 1000
	hF + hJ <= 1
	hB + mL <= 1
	mA + mD <= 1
	hB + hF + hG + hJ + hP + hS + mA + mC + mD + mL + mM = 5
	hB, hF, hG, hJ, hP, hS, mA, mC, mD, mL, mM ∈ [0,1]

Status: Sucesso, solução ótima encontrada
Valor objetivo (z): 452.0

Valores encontrados:
hB: 0.0
hF: 0.0
hG: 0.0
hJ: 1.0
hP: 0.0
hS: 1.0
mA: 0.0
mC: 0.0
mD: 1.0
mL: 1.0
mM: 1.0
